<a href="https://colab.research.google.com/github/jimxs74/SC/blob/main/SC62_LembarKoordinasi_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **SC62 Script Lembar Koordinasi**

Tool ini berguna untuk mendapatkan gambaran progress overall SC project secara angka. agar bisa diidentifikasi letak work volume terbanyak ada diSystem/Disiplin mana

UI/UX belum menjadi objective dari tool ini, akan dilanjutkan dengan framework yang lain

In [201]:
import pandas as pd
import numpy as np
import gspread

In [202]:
from gspread_dataframe import set_with_dataframe
from google.colab import auth
auth.authenticate_user()

import gspread
from gspread_dataframe import set_with_dataframe
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [203]:
def read_data(url, sheet='sheet1'):
  gs = gc.open_by_url(url).worksheet(sheet).get_all_values()
  gs = pd.DataFrame.from_records(gs)
  gs.columns = gs.iloc[0]
  gs = gs[1:]
  return gs

def write_data(url, result, sheet):
  write = gc.open_by_url(url)
  write = write.worksheet(sheet)
  write.clear()
  set_with_dataframe(worksheet=write, dataframe=result, include_index=False,include_column_header=True, resize=True)
  return

In [204]:
# gsheet yang akan diolah dan analisis
url_sc18 = 'https://docs.google.com/spreadsheets/d/1wrvBoSNka1f51hk8QuLd_bF7irVzSB6nDpg-Z_XqcG4/edit#gid=532748829'
url_cert = "https://docs.google.com/spreadsheets/d/1xPQanRi1XDmo7FlUyiV6Rt8pro_zjEHLOE0GLROTReQ/edit#gid=1035965456"
url_itr = "https://docs.google.com/spreadsheets/d/1aB57SwMedoMSjBxt5w_EBNYgmMoUACTJC056gEbD9a0/edit#gid=1491426671"
url_cc = "https://docs.google.com/spreadsheets/d/1ZrQjq30E58bFFiuuZ9aqAYy2StB4uKDRb2oDtM3m8wk/edit#gid=1688767959"
url_pc = "https://docs.google.com/spreadsheets/d/1DHwpGa_I0sd42o9zfC5cUMSud9XfKwPtyYm7UHY9xl0/edit#gid=1494113475"
url_pl = "https://docs.google.com/spreadsheets/d/19VIlyxleK14LjZCPls-4F9fjg0Sh46Q8oTk5UawJzhI/edit#gid=622809212"
url_sr = "https://docs.google.com/spreadsheets/d/1X3Zy083zX2spgZWqlw9TfnzUpHh_LR_C3H1jXyEdB4A/edit#gid=253348165"
url_com = 'https://docs.google.com/spreadsheets/d/1A2BrD-4wUNdJtoQmhoM2kxJD_yUPC9GcajIMdZWu9w0'

# Output tampilan
url_sc62 = 'https://docs.google.com/spreadsheets/d/1BRdjDg45HbNN1oeN22UwpaBwTL7fDTqOkMTwBHs0g5g/edit#gid=2092045117'

#dummy Test
#url_sc62 = 'https://docs.google.com/spreadsheets/d/1nO9qBfJCS8vH7shuc_BAj9OGNsdjwqXkhBhFvhNdAcI/edit#gid=0'

Buat tabel system owner dan subsystem owner

In [205]:
gs_read = gc.open_by_url(url_sc18).worksheet('System List')
gs_write = gc.open_by_url(url_sc62)

df = gs_read.get_all_values()
df = pd.DataFrame.from_records(df)

# Ambil kolom yg di perlukan saja dari SC18
df = df[[1,2,3,4,24,32]] 
df.columns = ['system','system desc','subsystem','subsystem desc', 'SSHOC or SHOC','system owner']
df['system'].replace('',np.nan, inplace=True)
df['subsystem'].replace('',np.nan, inplace=True)

# pilih row yang diperlukan saja
so = df[6:].drop(['subsystem','subsystem desc'], axis=1)

# drop row yg mengandung blank pada column subsystem
so.dropna(subset=['system'], inplace=True)
so.reset_index(drop=True)

# Generate summary table Sub-System Owner dari SC18
# pilih row yang diperlukan saja
sso = df[6:].drop(['system','system desc'], axis=1)

# drop row yg mengandung blank pada column subsystem
sso.dropna(subset=['subsystem'], inplace=True)
sso.reset_index(drop=True)


,subsystem,subsystem desc,SSHOC or SHOC,system owner
0,009-01,Temporary Power MV (20/11kV) and LV (400V),SSHOC,Yadi
1,009-02,Temporary Water Supply,SSHOC,Fathur Rohman
2,009-03,Temporary Safety and Fire Equipment,SSHOC,Hasan H
3,009-04,Temporary HVAC,,Fathur Rohman
4,009-06,Temporary Buyback Gas,SSHOC,Estu YP
...,...,...,...,...
972,870-23,Fire Area-12 (Substation-3),SSHOC,Noprian
973,870-24,Fire Area-14 (Substation-4),SSHOC,Noprian
974,870-25,Fire Area-15 (Control Building),SSHOC,
975,999-01,,,


In [206]:
so_sso = pd.concat([so, sso], axis=1)

In [207]:
so.columns = ['system/subsystem','desc', 'SSHOC/SHOC','system owner']
sso.columns = ['system/subsystem','desc', 'SSHOC/SHOC','system owner']
so_sso = pd.concat([so, sso])

In [208]:
'''
#write tabel so dan sso tabel ini tidak perlu ditampilkan dalam frontend gsheet
write_data(url_sc62, so, 'so')
write_data(url_sc62, sso, 'sso')
write_data(url_sc62, so_sso, 'so_sso')
'''

"\n#write tabel so dan sso tabel ini tidak perlu ditampilkan dalam frontend gsheet\nwrite_data(url_sc62, so, 'so')\nwrite_data(url_sc62, sso, 'sso')\nwrite_data(url_sc62, so_sso, 'so_sso')\n"

#1. Connect ke Data

In [209]:
'''
# link data dari p Supri. di update harian oleh p Supri
cert = read_data(url_cert,'CERT')
itr = read_data(url_itr,'ITR')
cc = read_data(url_cc,'CC')
pc = read_data(url_pc,'PC')
pl = read_data(url_pl,'PL')
sr = read_data(url_sr,'PHASE')
com = read_data(url_com,'COM')
'''

"\n# link data dari p Supri. di update harian oleh p Supri\ncert = read_data(url_cert,'CERT')\nitr = read_data(url_itr,'ITR')\ncc = read_data(url_cc,'CC')\npc = read_data(url_pc,'PC')\npl = read_data(url_pl,'PL')\nsr = read_data(url_sr,'PHASE')\ncom = read_data(url_com,'COM')\n"

In [210]:
# tabel prioritas
# Mengambil data dari SC62 worksheet priority dan clean up
priority = read_data(url_sc62,'priority')
priority = priority[['subsystem','phase_2']]
priority = priority.loc[priority['phase_2'].isin(['phase_1','phase_2'])]
priority = priority.rename(columns={"phase_2": "priority"})

#2. Spread = Certificate + so_sso

In [211]:
cert = read_data(url_cert,'CERT')

In [212]:
#ambil data certificate
spread = cert[['Certificate No','System No','Sub System No','Sub System Description','Discipline','Status']]

#merge dengan prioritas
spread = pd.merge(spread, priority, left_on ="Sub System No", right_on="subsystem" , how="left").drop(columns=['subsystem'])


In [213]:
#merge dg so_sso
spread = pd.merge(spread, so_sso, left_on ="System No", right_on="system/subsystem" , how="left")
spread = pd.merge(spread, so_sso, left_on ="Sub System No", right_on="system/subsystem" , how="left")

spread = pd.concat([spread.loc[spread['Sub System No'] == ''],spread.loc[spread['Sub System No'] != '']])
spread = spread.drop(['system/subsystem_x','desc_x','SSHOC/SHOC_x','system/subsystem_y','desc_y','SSHOC/SHOC_y','system owner_y'], axis=1).sort_values(by=['System No','Sub System No'])

spread = spread.reindex(columns=['System No','Sub System No', 'Sub System Description','Certificate No','Discipline','system owner_x','Status','priority'])

In [214]:
#write_data(url_sc62, spread, 'spread')

#2. Spread = spread + itr

In [215]:
itr = read_data(url_itr,'ITR')

In [216]:
#Ambil data ITR dan clean up
itr = itr[['Sub System No','Discipline','Tag No','ITR No','ITR Description','Status']]
# ambil hanya yang outstanding
itr = itr.loc[(itr['Status']=='Outstanding')]

# Membuat buffer percabangan untuk membuat tabel worksheet remaining ITR, utk digunakan nanti
itr_show = itr.drop(columns=['Status'])

# Menghitung jumlah ITR per subsystem
itr = itr.groupby(['Sub System No','Discipline'])['ITR No'].count().reset_index(name="jumlah")

# Combine text Discipline/Category/Jumlah C:1
itr["Discipline"] = itr["Discipline"].astype(str) + ":" + itr["jumlah"].astype(str)
itr = itr.drop(columns=['jumlah'])
itr = itr.set_index('Sub System No')

# combine value as string
itr = itr.assign(labels = itr.groupby(level = 'Sub System No').cumcount()).groupby([itr.index,'labels']).first().unstack('labels').sort_index(axis =1,level = 1).droplevel(1,axis = 1)
itr.head(3)

,Discipline,Discipline,Discipline,Discipline,Discipline,Discipline,Discipline,Discipline,Discipline
Sub System No,,,,,,,,,
009-06,L:1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
062-04,C:6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
063-01,C:4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [217]:
itr.columns = ['ITR','A','B','C','D','E','F','G','H']

itr["ITR"] = itr["ITR"].astype(str) + "/" + itr["A"].astype(str) + "/" + itr["B"].astype(str) + "/" + itr["C"].astype(str) + "/" + itr["D"].astype(str) + "/" + itr["E"].astype(str) + "/" + itr["F"].astype(str) + "/" + itr["G"].astype(str) + "/" + itr["H"].astype(str)
itr = itr.drop(columns=['A','B','C','D','E','F','G','H'])

# cleaning /nan/
itr['ITR'] = itr['ITR'].str.replace(r'nan[/]','', regex=True).str.replace(r'/nan','', regex=True)

In [218]:
'''
itr.columns = ['ITR','A','B','C','D','E','F','G','H','I']

itr["ITR"] = itr["ITR"].astype(str) + "/" + itr["A"].astype(str) + "/" + itr["B"].astype(str) + "/" + itr["C"].astype(str) + "/" + itr["D"].astype(str) + "/" + itr["E"].astype(str) + "/" + itr["F"].astype(str) + "/" + itr["G"].astype(str) + "/" + itr["H"].astype(str) + "/" + itr["I"].astype(str)
itr = itr.drop(columns=['A','B','C','D','E','F','G','H','I'])

# cleaning /nan/
itr['ITR'] = itr['ITR'].str.replace(r'nan[/]','', regex=True).str.replace(r'/nan','', regex=True)
'''

'\nitr.columns = [\'ITR\',\'A\',\'B\',\'C\',\'D\',\'E\',\'F\',\'G\',\'H\',\'I\']\n\nitr["ITR"] = itr["ITR"].astype(str) + "/" + itr["A"].astype(str) + "/" + itr["B"].astype(str) + "/" + itr["C"].astype(str) + "/" + itr["D"].astype(str) + "/" + itr["E"].astype(str) + "/" + itr["F"].astype(str) + "/" + itr["G"].astype(str) + "/" + itr["H"].astype(str) + "/" + itr["I"].astype(str)\nitr = itr.drop(columns=[\'A\',\'B\',\'C\',\'D\',\'E\',\'F\',\'G\',\'H\',\'I\'])\n\n# cleaning /nan/\nitr[\'ITR\'] = itr[\'ITR\'].str.replace(r\'nan[/]\',\'\', regex=True).str.replace(r\'/nan\',\'\', regex=True)\n'

In [219]:
# merge dengan table di sert_sso (worksheet sc_workfront)
spread = pd.merge(spread, itr, left_on ="Sub System No", right_on="Sub System No" , how="left")

In [220]:
#write_data(url_sc62, spread, 'spread')

#3. Spread = spread + summary report

ada bug kadang ada blank kadang ada data 0. perlu di buat dalam blank semua

In [221]:
sr = read_data(url_sr,'PHASE')

In [222]:
sr = sr[['System No', 'System Description', 'Sub System No','Sub System Description', 'Discipline', 'ITR Outstanding', 'CC Checksheet Outstanding', 'PC Checksheet Outstanding', 'COMM Checksheet Outstanding']]
#sr = sr.fillna(0)

sr['System No'] = pd.to_numeric(sr['System No'], errors='coerce').astype('Int64')
sr['ITR Outstanding'] = pd.to_numeric(sr['ITR Outstanding'], errors='coerce').astype('Int64')
sr['CC Checksheet Outstanding'] = pd.to_numeric(sr['CC Checksheet Outstanding'], errors='coerce').astype('Int64')
sr['PC Checksheet Outstanding'] = pd.to_numeric(sr['PC Checksheet Outstanding'], errors='coerce').astype('Int64')
sr['COMM Checksheet Outstanding'] = pd.to_numeric(sr['COMM Checksheet Outstanding'], errors='coerce').astype('Int64')

sr = sr.fillna(0)

In [223]:
#sr['CC Checksheet Outstanding'] = sr['CC Checksheet Outstanding'].replace({'0':np.nan, 0:np.nan})
#sr = sr.replace({'0':np.nan, 0:np.nan})

In [224]:
# Filter yg masih outstanding saja di bagian CC Checksheet
sr_cc = sr[['Sub System No','Discipline','CC Checksheet Outstanding']]

# combine text on different column
sr_cc["Discipline"] = sr_cc["Discipline"].astype(str) + ":" + sr_cc["CC Checksheet Outstanding"].astype(str)
sr_cc = sr_cc.drop(columns=['CC Checksheet Outstanding'])
sr_cc = sr_cc.set_index('Sub System No')

# combine value as string ref : https://stackoverflow.com/questions/59914167/putting-all-the-same-value-indices-into-one-row
sr_cc = sr_cc.assign(labels = sr_cc.groupby(level = 'Sub System No').cumcount()).groupby([sr_cc.index,'labels']).first().unstack('labels').sort_index(axis =1,level = 1).droplevel(1,axis = 1)

# sering ada bug disini, dibuat complete biar tidak ada bug
sr_cc.columns = ['CC', 'A','B','C','D','E']

sr_cc["CC"] = sr_cc["CC"].astype(str) + "/" + sr_cc["A"].astype(str) + "/" + sr_cc["B"].astype(str) + "/" + sr_cc["C"].astype(str) + "/" + sr_cc["D"].astype(str) + "/" + sr_cc["E"].astype(str)
sr_cc = sr_cc.drop(columns=['A','B','C','D','E'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [225]:
sr_pc = sr[['Sub System No','Discipline','PC Checksheet Outstanding']]

# combine text on different column
sr_pc["Discipline"] = sr_pc["Discipline"].astype(str) + ":" + sr_pc["PC Checksheet Outstanding"].astype(str)
sr_pc = sr_pc.drop(columns=['PC Checksheet Outstanding'])
sr_pc = sr_pc.set_index('Sub System No')

# combine value as string 
sr_pc = sr_pc.assign(labels = sr_pc.groupby(level = 'Sub System No').cumcount()).groupby([sr_pc.index,'labels']).first().unstack('labels').sort_index(axis =1,level = 1).droplevel(1,axis = 1)

#adjustment jumlah column A/B/C/D/E
sr_pc.columns = ['PC', 'A','B','C','D','E']
sr_pc["PC"] = sr_pc["PC"].astype(str) + "/" + sr_pc["A"].astype(str) + "/" + sr_pc["B"].astype(str) + "/" + sr_pc["C"].astype(str) + "/" + sr_pc["D"].astype(str) + "/" + sr_pc["E"].astype(str)
sr_pc = sr_pc.drop(columns=['A','B','C','D','E'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [226]:
# merge cc_pc
cc_pc = pd.merge(sr_cc, sr_pc, left_on ="Sub System No", right_on="Sub System No" , how="outer")
# perlu ada fungsi untuk cleaning /nan/
cc_pc['CC'] = cc_pc['CC'].str.replace(r'nan[/]','', regex=True).str.replace(r'/nan','', regex=True)
cc_pc['PC'] = cc_pc['PC'].str.replace(r'nan[/]','', regex=True).str.replace(r'/nan','', regex=True)

cc_pc['CC'] = cc_pc['CC'].str.replace(r'[A-Z]:0/','', regex=True).str.replace(r'/[A-Z]:0','', regex=True).str.replace(r'[A-Z]:0','', regex=True)
cc_pc['PC'] = cc_pc['PC'].str.replace(r'[A-Z]:0/','', regex=True).str.replace(r'/[A-Z]:0','', regex=True).str.replace(r'[A-Z]:0','', regex=True)

In [227]:
spread = pd.merge(spread, cc_pc, left_on ="Sub System No", right_on="Sub System No" , how="left")

In [228]:
#write_data(url_sc62, spread, 'spread')

#5. Connect to PunchList Data (cert+sso+summary report + punch)

In [229]:
pl = read_data(url_pl,'PL')

In [230]:
# select only useful column
pl = pl[['Sub System No','Discipline','Category','Punchlist No','Punchlist Description','Status','Phase','Raised By']] 
pl = pl.loc[pl['Status']=='Outstanding'].drop(columns=['Status']) # hemat data yg outstanding saja yg di olah

#membuat buffer percabangan untuk di buat table dedicated dalam worksheet pl_A dan pl_B
pl_a = pl.loc[pl['Category'].isin(['A'])]
pl_b = pl.loc[pl['Category'].isin(['B'])]
pl_c = pl.loc[pl['Category'].isin(['C'])]

pl_ab = pl.loc[pl['Category'].isin(['A','B'])]   # saat include C, line ini akan di escape



In [231]:
# Menghitung jumlah punch saja
pl_ab = pl_ab.groupby(['Sub System No','Discipline','Category'])['Punchlist No'].count().reset_index(name="jumlah")

# Combine text Discipline/Category/Jumlah C:(B=4)
pl_ab["Discipline"] = pl_ab["Discipline"].astype(str) + ":(" + pl_ab["Category"].astype(str) + "=" + pl_ab["jumlah"].astype(str) + ")"

pl_ab = pl_ab.drop(columns=['Category','jumlah'])
pl_ab = pl_ab.set_index('Sub System No')

# combine value as string ref
pl_ab = pl_ab.assign(labels = pl_ab.groupby(level = 'Sub System No').cumcount()).groupby([pl_ab.index,'labels']).first().unstack('labels').sort_index(axis =1,level = 1).droplevel(1,axis = 1)

In [232]:
pl_ab.sample(3)

,Discipline,Discipline,Discipline,Discipline,Discipline,Discipline,Discipline
Sub System No,,,,,,,
567-20,E:(A=4),NaN,NaN,NaN,NaN,NaN,NaN
590-03,E:(A=7),E:(B=2),I:(A=1),I:(B=5),NaN,NaN,NaN
492-15,S:(B=1),NaN,NaN,NaN,NaN,NaN,NaN


In [233]:
#jika jumlah column exclude index ada 6
#pl_ab.columns = ['Punch', 'A','B','C','D','E'] 
#pl_ab["Punch"] = pl_ab["Punch"].astype(str) + "/" + pl_ab["A"].astype(str) + "/" + pl_ab["B"].astype(str) + "/" + pl_ab["C"].astype(str) + "/" + pl_ab["D"].astype(str) + "/" + pl_ab["E"].astype(str)
#pl_ab = pl_ab.drop(columns=['A','B','C','D','E'])# jika ada error jumlah element

#jika jumlah column exclude index ada 7
pl_ab.columns = ['Punch', 'A','B','C','D','E','F'] # jika ada error jumlah element
pl_ab["Punch"] = pl_ab["Punch"].astype(str) + "/" + pl_ab["A"].astype(str) + "/" + pl_ab["B"].astype(str) + "/" + pl_ab["C"].astype(str) + "/" + pl_ab["D"].astype(str) + "/" + pl_ab["E"].astype(str) + "/" + pl_ab["F"].astype(str)
pl_ab = pl_ab.drop(columns=['A','B','C','D','E','F'])# jika ada error jumlah element

#jika jumlah column exclude index ada >7
#pl_ab.columns = ['Punch', 'A','B','C','D','E','F','G','H','I']
#pl_ab["Punch"] = pl_ab["Punch"].astype(str) + "/" + pl_ab["A"].astype(str) + "/" + pl_ab["B"].astype(str) + "/" + pl_ab["C"].astype(str) + "/" + pl_ab["D"].astype(str) + "/" + pl_ab["E"].astype(str) + "/" + pl_ab["F"].astype(str) + "/" + pl_ab["G"].astype(str)
#pl_ab = pl_ab.drop(columns=['A','B','C','D','E','F','G','H','I'])

pl_ab['Punch'] = pl_ab['Punch'].str.replace(r'nan[/]','', regex=True).str.replace(r'/nan','', regex=True)


In [234]:
# Merge dengan data 
spread = pd.merge(spread, pl_ab, left_on ="Sub System No", right_on="Sub System No" , how="left")

In [235]:
# tampilkan yg outstanding saja
spread = spread.loc[(spread['Status']=='Outstanding')] # uncomment me, jika utk melihat keseluruhan data

In [236]:
write_data(url_sc62, spread, 'spread')

# Fungsi check kekompletan certificate

In [237]:
a = spread['System No']
b = spread['Sub System No']
f = spread['system owner_x']
i = spread['ITR']
j = spread['CC']
k = spread['PC']
l = spread['Punch']

In [264]:
# membuat logic apakah ITR tinggal Piping insulasi saja?
'''
logic_1 = level subsytem, ada system owner, CC=0, PC=0. punch = 0, ITR = 0  -> Issue Cert !!!
logic_2 = level subsytem, ada system owner, CC=0, PC=0. punch = 0, ITR = mempunyai string *L:*  -> itr paint/insul kah? !!
logic_3 = ~(logic_1 or logic_2)
'''
logic_1 = (b != '') & (f != '') & (j == '') & (k == '') & (l == '') & (i == '')
logic_2 = (b != '') & (f != '') & (j == '') & (k == '') & (l == '') & (i.str.contains(pat='L:'))
logic_3 = ~(logic_1 | logic_2)

condition = [
    logic_1,
    logic_2,
    logic_3
]
result = ['sudah WD?yes -> Issue Cert !!','itr paint/insul?yes -> issue Cert !!', '']

spread['autocheck'] = np.select(condition, result)
write_data(url_sc62, spread, 'spread')


#8. Refresh List item Outstanding ITR/PC/CC/ITR

In [239]:
itr = read_data(url_itr,'ITR')

#Ambil data ITR dan clean up
itr = itr[['Sub System No','Discipline','Tag No','ITR No','ITR Description','Status']]
# ambil hanya yang outstanding
itr = itr.loc[(itr['Status']=='Outstanding')]

# Membuat buffer percabangan untuk membuat tabel worksheet remaining ITR, utk digunakan nanti
itr_show = itr.drop(columns=['Status'])

In [240]:
# merge dengan table priority 
itr_show = pd.merge(itr_show, priority , left_on ="Sub System No", right_on="subsystem" , how="left")
itr_show = itr_show.loc[itr_show['priority'].isin(['phase_1','phase_2'])]
itr_show = itr_show.drop(columns=["subsystem"])

# Update worksheet remaining ITR 
write_data(url_sc62, itr_show, 'itr')

In [241]:
'''
#drop row 'ITR No' yg blank
itr_show['ITR No'].replace('', np.nan, inplace=True)
itr_show.dropna(subset=['ITR No'], inplace=True)
'''

"\n#drop row 'ITR No' yg blank\nitr_show['ITR No'].replace('', np.nan, inplace=True)\nitr_show.dropna(subset=['ITR No'], inplace=True)\n"

cc

In [242]:
cc = read_data(url_cc,'CC')

In [243]:
#Ambil data CC dan clean up
cc = cc[['Sub System No','Discipline','Tag No','Checksheet Description','Status']]

# ambil hanya yang outstanding
cc = cc.loc[(cc['Status']=='Outstanding')]
cc = cc.drop(columns=['Status'])

# merge dengan table phase_2
cc = pd.merge(cc, priority, left_on ="Sub System No", right_on="subsystem" , how="left")

cc = cc.loc[cc['priority'].isin(['phase_1','phase_2'])]

# Update worksheet remaining CC
write_data(url_sc62, cc, 'cc')

pl_a

In [244]:
pl_a = pl.loc[pl['Category'].isin(['A'])]

In [245]:
# merge dengan table phase_2
pl_a = pd.merge(pl_a, priority, left_on ="Sub System No", right_on="subsystem" , how="left")
# hanya tunjukan yg ada dalam jalur phase_2
pl_a = pl_a.loc[pl_a['priority'].isin(['phase_1','phase_2'])]
pl_a = pl_a.drop(columns=["subsystem"])

In [246]:
# Update worksheet remaining PL-A
write_data(url_sc62, pl_a, 'pl_a')

pc

In [247]:
pc = read_data(url_pc,'PC')

In [248]:
#Ambil data PC dan clean up
pc = pc[['Sub System No','Discipline','Tag No','Checksheet No','Checksheet Description','Status']]

# ambil hanya yang outstanding
pc = pc.loc[(pc['Status']=='Outstanding')]
pc = pc.drop(columns=['Status'])

In [249]:
# merge dengan table phase_2
pc = pd.merge(pc, priority, left_on ="Sub System No", right_on="subsystem" , how="left")

In [250]:
# hanya tunjukan yg ada dalam jalur phase_2
pc = pc.loc[pc['priority'].isin(['phase_1','phase_2'])]
pc = pc.drop(columns=["subsystem"])

In [251]:
# Update worksheet remaining PC # di pindahkan ke bawah
#write_data(url_sc62, pc, 'pc')

pl_b

In [252]:
pl_b = pl.loc[pl['Category'].isin(['B'])]

In [253]:
# merge dengan table phase_2
pl_b = pd.merge(pl_b, priority, left_on ="Sub System No", right_on="subsystem" , how="left")
# hanya tunjukan yg ada dalam jalur phase_2
pl_b = pl_b.loc[pl_b['priority'].isin(['phase_1','phase_2'])]
pl_b = pl_b.drop(columns=["subsystem"])

In [254]:
# Update worksheet remaining PL-B
write_data(url_sc62, pl_b, 'pl_b')

comm

In [255]:
comm = read_data(url_com,'COM')

In [256]:
#Ambil data comm dan clean up
comm = comm[['System No','System Description','Sub System No','Checksheet No','Status']]

# ambil hanya yang outstanding
comm = comm.loc[(comm['Status']=='Outstanding')]
comm = comm.drop(columns=['Status'])

In [257]:
# Update worksheet remaining PC # di pindahkan ke bawah
write_data(url_sc62, comm, 'comm')

pl_2nd, akan dipakai saat arah diskusi sampai dengan prioritas 2

In [258]:
# re-input data/subject utk di delete
pl_c = pl.loc[pl['Category'].isin(['C'])]
pl_ab = pl.loc[pl['Category'].isin(['A','B'])]

In [259]:
pl_ab_2nd = pd.merge(pl_ab, priority, left_on ="Sub System No", right_on="subsystem" , how="left")

# invert pemilihan pl_ab yg tidak masuk dalam prioritas
pl_ab_2nd = pl_ab_2nd.loc[~pl_ab_2nd['priority'].isin(['phase_1','phase_2'])]
pl_ab_2nd = pl_ab_2nd.drop(columns=["subsystem","priority"])

#diconcatinasi
pl_2nd = pd.concat([pl_ab_2nd,pl_c])

# Update worksheet remaining PL-C 
#write_data(url_sc62, pl_2nd, 'pl_2nd') # normaly tidak aktif utk speed, jika perlu muncul di SC61 uncomment

#9. Connect to Transmital SC17 (pc dan certificate)

pc

In [260]:
# terhubung dengan SC17
url_sc17 = 'https://docs.google.com/spreadsheets/d/1nrdtZOiToC_kW95a8rKR_-J8hH5Io5Rp_JG2a8S2T3o/edit#gid=1274212197'

sc17_cert = read_data(url_sc17,'Certificate Monitoring')
sc17_pc = read_data(url_sc17,'PC Monitoring')
sc17_cc = read_data(url_sc17,'CS Monitoring ')

In [261]:
#ambil data transmital PC yg diperlukan dan clean up
sc17_cc = sc17_cc[['Subsystem','Date','   Discipline','Tag No','Checksheet No','Status','Remarks','Transmital No']]
#merge data
cc_x1 = pd.merge(cc, sc17_cc, left_on ="Tag No", right_on="Tag No" , how="left")

#di atur kolomnya
cc_x1 = cc_x1.reindex(columns=['Sub System No','Discipline', 'Tag No','Checksheet Description', 'subsystem desc','Date','Status','Remarks','Transmital No'])
cc_x1 = cc_x1.rename(columns={"Date": "SC17_submit Date", "Remarks": "SC17 Status"})

# write cc
#write_data(url_sc62, cc_x1, 'cc')

In [262]:
#ambil data transmital PC yg diperlukan dan clean up
sc17_pc = sc17_pc[['Subsystem No.','Date','Discipline','Tag No.','Checksheet No','Checksheet Description','Status','Remarks','Transmital No']]

#merge data
#pc_x1 = pd.merge(pc, sc17_pc, left_on ="Tag No", right_on="Tag No." , how="left") # ada merging yg kurang pas, karena tag number tidak unique di SC17
pc_x1 = pd.merge(pc, sc17_pc, left_on =["Tag No","Checksheet No"], right_on=["Tag No.","Checksheet No"] , how="left") # absorbing comment/finding bug p Adang, perlu check 2 column


#di atur kolomnya
pc_x1 = pc_x1.reindex(columns=['Sub System No','Discipline_x', 'Tag No','Checksheet No', 'Checksheet Description_x','Date','Remarks','Transmital No'])
pc_x1 = pc_x1.rename(columns={"Date": "SC17_submit Date", "Remarks": "SC17 Status"})

# write pc
write_data(url_sc62, pc_x1, 'pc')

certificate

In [263]:
'''
sc17_cert = read_data(url_sc17,'Certificate Monitoring')

#ambil data transmital cert yg diperlukan dan clean up
#sc17_cert = sc17_cert[['System/sub-system','Date','Discipline','Certificate','Re-submit','Remarks','Transmital No']]
sc17_cert = sc17_cert[['Sub-System No.','Date','Discipline','Certificate','Submit','Remarks','Transmital No']]

#merge data
spread_1 = pd.merge(spread, sc17_cert, left_on =["Tag No","Checksheet No"], right_on=["Tag No.","Checksheet No"] , how="left") # absorbing comment/finding bug p Adang, perlu check 2 column

# write pc
write_data(url_sc62, sc17_cert, 'test')

'''

'\nsc17_cert = read_data(url_sc17,\'Certificate Monitoring\')\n\n#ambil data transmital cert yg diperlukan dan clean up\n#sc17_cert = sc17_cert[[\'System/sub-system\',\'Date\',\'Discipline\',\'Certificate\',\'Re-submit\',\'Remarks\',\'Transmital No\']]\nsc17_cert = sc17_cert[[\'Sub-System No.\',\'Date\',\'Discipline\',\'Certificate\',\'Submit\',\'Remarks\',\'Transmital No\']]\n\n#merge data\nspread_1 = pd.merge(spread, sc17_cert, left_on =["Tag No","Checksheet No"], right_on=["Tag No.","Checksheet No"] , how="left") # absorbing comment/finding bug p Adang, perlu check 2 column\n\n# write pc\nwrite_data(url_sc62, sc17_cert, \'test\')\n\n'